In [1]:
!pip install hyperopt


  Created wheel for networkx: filename=networkx-2.2-py2.py3-none-any.whl size=1527327 sha256=f6470d728907f177125b40de8ff60e4ffb92f0428e1b9c30c64a98913f12af35
  Stored in directory: C:\Users\benia\AppData\Local\pip\Cache\wheels\68\f8\29\b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
Successfully built networkx
  Found existing installation: networkx 2.3
    Uninstalling networkx-2.3:
      Successfully uninstalled networkx-2.3


In [5]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import eli5 
from eli5.sklearn import PermutationImportance
import xgboost as xgb
from hyperopt import hp,fmin,tpe,STATUS_OK

In [11]:
cd data

C:\Users\benia\Desktop\Machine_Learning\Prediction price cars\data


In [12]:
df = pd.read_hdf('car.h5')

In [13]:
SUFFIX_CAT = "_cat"
for feat in df.columns:
    if isinstance(df[feat][0],list): continue
    
    factorized_values = df[feat].factorize()[0]
    
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat + SUFFIX_CAT] = factorized_values


In [14]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [15]:
def run_model(model , feats):
    X = df[feats].values
    y = df["price_value"].values
    
    scores = cross_val_score(model,X,y,cv=3,scoring="neg_mean_absolute_error")
    return np.mean(scores), np.std(scores)

In [16]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== "None" else int(x))
df['param_moc']= df['param_moc'].map(lambda x: -1 if str(x)== "None" else int(x.split(' ')[0]))
df['param_pojemność-skokowa']= df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)== "None" else int(str(x).split('cm')[0].replace(' ','')))

In [22]:
feats =['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_faktura-vat_cat','param_moc','param_skrzynia-biegów_cat','feature_kamera-cofania_cat','param_marka-pojazdu_cat','param_typ_cat','param_pojemność-skokowa','feature_wspomaganie-kierownicy_cat','seller_name_cat','param_wersja_cat','param_model-pojazdu_cat','feature_światła-led_cat','feature_asystent-pasa-ruchu_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_regulowane-zawieszenie_cat','feature_łopatki-zmiany-biegów_cat']
run_model(xgb.XGBRegressor(**xgb_params),feats)

(-9618.08167109244, 79.8982870999676)

In [18]:
xgb_params={
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

In [21]:
def obj_func(params):
    print('Training with params:')
    print(params)
    
    mean_mae , score_std = run_model(xgb.XGBRegressor(**xgb_params),feats)
    
    return {'loss':np.abs(mean_mae),'status':STATUS_OK}

xgb_reg_params={
    'learning_rate':    hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth':        hp.choice('max_depth',     np.arange(5,16,1,dtype=int)),
    'subsample':        hp.quniform('subsample',0.5,1,0.05),   
    'colsample_bytree':  hp.quniform('colsample_bytree',0.5,1,0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':0,}
best = fmin(obj_func,xgb_reg_params,algo=tpe.suggest,max_evals=25)
best

Training with params:                                                                                                  
{'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params:                                                                                                  
{'colsample_bytree': 0.75, 'learning_rate': 0.15000000000000002, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params:                                                                                                  
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params:                                                                                                  
{'colsample_bytree': 0.55, 'learning_rate'

{'colsample_bytree': 0.75,
 'learning_rate': 1,
 'max_depth': 6,
 'subsample': 0.55}

In [23]:
def obj_func(params):
    print('Training with params:')
    print(params)
    
    mean_mae , score_std = run_model(xgb.XGBRegressor(**xgb_params),feats)
    
    return {'loss':np.abs(mean_mae),'status':STATUS_OK}

xgb_reg_params={
    'learning_rate':    hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth':        hp.choice('max_depth',     np.arange(5,16,1,dtype=int)),
    'subsample':        hp.quniform('subsample',0.5,1,0.05),   
    'colsample_bytree':  hp.quniform('colsample_bytree',0.5,1,0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':0,}
best = fmin(obj_func,xgb_reg_params,algo=tpe.suggest,max_evals=50)
best

Training with params:                                                                                                  
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params:                                                                                                  
{'colsample_bytree': 0.55, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params:                                                                                                  
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params:                                                                                                  
{'colsample_by

{'colsample_bytree': 0.5, 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params:                                                                                                  
{'colsample_bytree': 0.9, 'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params:                                                                                                  
{'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params:                                                                                                  
{'colsample_bytree': 0.5, 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with pa

{'colsample_bytree': 0.8500000000000001,
 'learning_rate': 4,
 'max_depth': 5,
 'subsample': 0.6000000000000001}

In [30]:
xgb_params={
        'n_estimators':50,
        'colsample_bytree': 0.8500000000000001,
        'learning_rate': 4,
        'max_depth': 5,
        'subsample': 0.6000000000000001
}
run_model(xgb.XGBRegressor(**xgb_params),feats)

(-3.2376552477165094e+28, 8.560970975533546e+25)

In [29]:
xgb_params={'colsample_bytree': 0.75,
 'learning_rate': 1,
 'max_depth': 6,
 'subsample': 0.55,
 'max_depth': 5,
 'n_estimators':50,}
run_model(xgb.XGBRegressor(**xgb_params),feats)

(-11475.08530472996, 164.11129655574533)